# Employee Attrition Analysis — EDA & Modeling

This notebook trains models to predict employee attrition and exports artifacts for the dashboard.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay
from joblib import dump
from src.preprocess import split_features_target, get_feature_groups, build_preprocessor, train_valid_split, basic_metrics, confusion

DATA_PATH = Path('data/synthetic_hr_attrition.csv')  # change to your CSV if needed
df = pd.read_csv(DATA_PATH)
print(df.head())
print(df.shape)

## Quick EDA

In [ ]:
df.describe(include='all').T.head(20)

In [ ]:
attrition_rate = (df['Attrition'].str.title()=='Yes').mean()
print('Overall Attrition Rate:', round(100*attrition_rate, 2), '%')

In [ ]:
# Attrition by Department
dept = df.groupby('Department')['Attrition'].apply(lambda s: (s.str.title()=='Yes').mean()).sort_values(ascending=False)
dept.plot(kind='bar'); plt.title('Attrition Rate by Department'); plt.ylabel('Rate'); plt.show()

## Preprocessing & Split

In [ ]:
X, y = split_features_target(df)
num_cols, cat_cols = get_feature_groups(df)
pre = build_preprocessor(num_cols, cat_cols)
X_train, X_test, y_train, y_test = train_valid_split(X, y)

## Train: Logistic Regression

In [ ]:
log_clf = Pipeline([('pre', pre), ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))])
log_clf.fit(X_train, y_train)
y_pred_log = log_clf.predict(X_test)
y_proba_log = getattr(log_clf, 'predict_proba', lambda X: None)(X_test)
metrics_log = basic_metrics(y_test, y_pred_log, y_proba_log[:,1] if y_proba_log is not None else None)
metrics_log

## Train: Random Forest

In [ ]:
rf_clf = Pipeline([('pre', pre), ('clf', RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced'))])
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
y_proba_rf = rf_clf.predict_proba(X_test)[:,1]
metrics_rf = basic_metrics(y_test, y_pred_rf, y_proba_rf)
metrics_rf

## Compare Models

In [ ]:
pd.DataFrame([metrics_log, metrics_rf], index=['LogReg','RandomForest'])

## Plots

In [ ]:
fig, ax = plt.subplots()
RocCurveDisplay.from_predictions(y_test, y_proba_rf, name='RF', ax=ax)
if isinstance(y_proba_log, np.ndarray):
    RocCurveDisplay.from_predictions(y_test, y_proba_log[:,1], name='LogReg', ax=ax)
plt.show()

## Export Best Model

In [ ]:
best = rf_clf if metrics_rf.get('roc_auc',0) >= metrics_log.get('roc_auc',0) else log_clf
dump(best, 'models/model.pkl')
# Export the fitted preprocessor as well (it's inside the Pipeline but saving separately is sometimes handy)
dump(pre, 'models/preprocessor.pkl')
print('Saved to models/model.pkl and models/preprocessor.pkl')